In [ ]:
import numpy as np
import pandas as pd

In [ ]:
movies = pd.read_csv('D:/Movie Recommend/data/tmdb_5000_movies.csv')
credits = pd.read_csv('D:/Movie Recommend/data/tmdb_5000_credits.csv')

In [ ]:
movies.head(1)
credits.head(1)

In [ ]:
movies=movies.merge(credits, on='title')

In [ ]:
movies.shape

In [ ]:
movies.columns

In [ ]:
movies=movies[['id','title','overview','genres','keywords','cast','crew']]

In [ ]:
movies.head(1)

In [ ]:
movies.isnull().sum()

In [ ]:
movies.dropna(inplace=True)

In [ ]:
movies.duplicated().sum()

In [ ]:
import ast

In [ ]:
def convert(obj):
    L=[]
    for i in ast.literal_eval(obj):
        L.append(i['name'])
    return L

In [ ]:
movies['genres']=movies['genres'].apply(convert)

In [ ]:
movies.head(1)

In [ ]:
movies['keywords']=movies['keywords'].apply(convert)

In [ ]:
def convert_cast(obj):
    L = []
    count = 0
    for i in ast.literal_eval(obj):
        if count < 3:
            L.append(i['name'])
            count += 1
        else:
            break
    return L

In [ ]:
movies['cast']=movies['cast'].apply(convert_cast)

In [ ]:
movies.head(1)

In [ ]:
def convert_crew(obj):
    L = []
    for i in ast.literal_eval(obj):
        if i['job']=='Director':
            L.append(i['name'])
    return L

In [ ]:
movies['crew']=movies['crew'].apply(convert_crew)

In [ ]:
movies.head(5)

In [ ]:
movies['overview']=movies['overview'].apply(lambda x:x.split())

In [ ]:
movies.head()

In [ ]:
movies['genres']=movies['genres'].apply(lambda x:[i.replace(" ","") for i in x])
movies['keywords']=movies['keywords'].apply(lambda x:[i.replace(" ","") for i in x])
movies['overview']=movies['overview'].apply(lambda x:[i.replace(" ","") for i in x])
movies['cast']=movies['cast'].apply(lambda x:[i.replace(" ","") for i in x])
movies['crew']=movies['crew'].apply(lambda x:[i.replace(" ","") for i in x])

In [ ]:
movies.head()

In [ ]:
movies['tag']=movies['overview']+movies['genres']+movies['keywords']+movies['cast']+movies['crew']

In [ ]:
new_data=movies[['id','title','tag']]

In [ ]:
new_data['tag']=new_data['tag'].apply(lambda x:" ".join(x))
new_data['tag']=new_data['tag'].apply(lambda x:x.lower())

In [ ]:
new_data.head()

In [ ]:
import nltk

In [ ]:
from nltk.stem.porter import PorterStemmer
ps=PorterStemmer()

In [ ]:
from sklearn.feature_extraction.text import CountVectorizer

In [ ]:
cv=CountVectorizer(max_features=5000,stop_words='english')

In [ ]:
def stem(text):
    y=[]
    for i in text.split():
        y.append(ps.stem(i))
    return " ".join(y)

In [ ]:
new_data['tag']=new_data['tag'].apply(stem)

In [ ]:
vectors=cv.fit_transform(new_data['tag']).toarray()

In [ ]:
vectors.shape

In [ ]:
from sklearn.metrics.pairwise import cosine_similarity

In [ ]:
similarity=cosine_similarity(vectors)

In [ ]:
def recommended(movie):
    movie_index=new_data[new_data['title']==movie].index[0]
    distances=similarity[movie_index]

    movies_list=sorted(list(enumerate(distances)),reverse=True,key=lambda x:x[1])[1:6]
    for i in movies_list:
        print(new_data.iloc[i[0]].title)

In [ ]:
movie="Avatar"
recommended(movie)

In [ ]:
import pickle

In [ ]:
pickle.dump(new_data.to_dict(),open('movie_dict.pkl','wb'))

In [ ]:
pickle.dump(similarity,open('similarity.pkl','wb'))